## Context:

* problem: optimization problem

* Should: Timing for charging and discharging i guess

* Objective: When to charge and discharge
* Constraints: - Physical and Commercial market attributes
* Decision Variables: Levers


## Battery Storage Information:

* Power Capacity (kW or MW) is the total possible instantaneous charge/discharge capability of the BESS
* Energy Capacity (kWh or MWh) is the maximum amount of stored energy in the BESS
* Storage Duration (h) is the number of hours BESS will take to discharge at its power capacity before depleting its energy capacity or vice versa
* State of Charge, SOC (%) represents the battery’s present level of charge and ranges from completely discharged (0%) to fully charged (100%)
* Charging efficiency (%) is the energy charged to the battery divided by the energy consumed from the grid
* Discharging efficiency (%) is the energy supplied to the grid divided by the energy discharged by the battery


## TBN:

sum of the top n priced hours in a day  subtract the sum of the bottom n priced hours in a day === the revenue you could generate from a BESS project with n hours of duration.

Limitations:

The TBn method does not ensure State of Charge (SOC) feasibility.
There may be instances where one of the T2 hours occurs when the battery has no energy, or B2 hours occur when the battery is at full charge.

In [149]:
import pandas as pd
import numpy as np

In [150]:
rtm_prices = pd.read_excel("https://github.com/aps0611/case_energy/raw/main/RTM_Prices_2022.xlsx")
dam_prices = pd.read_excel("https://github.com/aps0611/case_energy/raw/main/DAM_Prices_2022.xlsx")

Lets explore the data:

In [151]:
rtm_prices.head(5)

,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point,Settlement Point Price
0,01/01/2022,1,1,N,HB_BUSAVG,52.77
1,01/01/2022,1,2,N,HB_BUSAVG,120.80
2,01/01/2022,1,3,N,HB_BUSAVG,41.02
3,01/01/2022,1,4,N,HB_BUSAVG,35.00
4,01/01/2022,1,1,N,HB_HOUSTON,52.82


In [152]:
rtm_prices.shape

(68448, 6)

In [153]:
rtm_prices.describe(include="all")

,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point,Settlement Point Price
count,68448,68448.000000,68448.000000,68448,68448,68448.000000
unique,31,NaN,NaN,1,15,NaN
top,01/01/2022,NaN,NaN,N,LZ_AEN,NaN
freq,2208,NaN,NaN,68448,5952,NaN
mean,NaN,12.500000,2.500000,NaN,NaN,29.909403
std,NaN,6.922237,1.118042,NaN,NaN,28.919321
min,NaN,1.000000,1.000000,NaN,NaN,-64.200000
25%,NaN,6.750000,1.750000,NaN,NaN,22.040000
50%,NaN,12.500000,2.500000,NaN,NaN,25.785000
75%,NaN,18.250000,3.250000,NaN,NaN,35.070000


In [154]:
## Delivery hour:

'''
min = 1    ---------------    00.00 midnight- assume
max = 24   ---------------    23.00 assume

'''

'\nmin = 1    ---------------    00.00 midnight- assume\nmax = 24   ---------------    23.00 assume\n\n'

In [155]:
rtm_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68448 entries, 0 to 68447
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Delivery Date           68448 non-null  object 
 1   Delivery Hour           68448 non-null  int64  
 2   Delivery Interval       68448 non-null  int64  
 3   Repeated Hour Flag      68448 non-null  object 
 4   Settlement Point        68448 non-null  object 
 5   Settlement Point Price  68448 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 3.1+ MB


In [156]:
def time_modification(delivery_hour):
    return f'{delivery_hour - 1:02d}'

In [157]:
time_modification(24)

'23'

In [158]:
rtm_prices['Delivery_Hour_new'] = rtm_prices['Delivery Hour'].apply(time_modification)

In [159]:
rtm_prices['Delivery_interval_new'] = (rtm_prices['Delivery Interval'] - 1) * 15

In [160]:
rtm_prices['dateTime'] = pd.to_datetime(rtm_prices['Delivery Date'] + ' ' + rtm_prices['Delivery_Hour_new'] + ':00:00') + pd.to_timedelta(rtm_prices['Delivery_interval_new'], unit='minutes')

In [161]:
rtm_prices.head(5)

,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point,Settlement Point Price,Delivery_Hour_new,Delivery_interval_new,dateTime
0,01/01/2022,1,1,N,HB_BUSAVG,52.77,00,0,2022-01-01 00:00:00
1,01/01/2022,1,2,N,HB_BUSAVG,120.80,00,15,2022-01-01 00:15:00
2,01/01/2022,1,3,N,HB_BUSAVG,41.02,00,30,2022-01-01 00:30:00
3,01/01/2022,1,4,N,HB_BUSAVG,35.00,00,45,2022-01-01 00:45:00
4,01/01/2022,1,1,N,HB_HOUSTON,52.82,00,0,2022-01-01 00:00:00


In [162]:
rtm_prices.dateTime.dtype

dtype('<M8[ns]')

In [163]:
dam_prices.head(5)

,Delivery Date,Delivery Hour,Repeated Hour Flag,Settlement Point,Settlement Point Price
0,01/01/2022,1,N,HB_BUSAVG,32.72
1,01/01/2022,1,N,HB_HOUSTON,33.20
2,01/01/2022,1,N,HB_HUBAVG,32.69
3,01/01/2022,1,N,HB_NORTH,33.41
4,01/01/2022,1,N,HB_PAN,31.28


In [164]:
dam_prices.shape

(11160, 5)

In [165]:
dam_prices.describe()

,Delivery Hour,Settlement Point Price
count,11160.000000,11160.000000
mean,12.500000,32.675469
std,6.922497,15.545797
min,1.000000,-2.270000
25%,6.750000,24.580000
50%,12.500000,29.220000
75%,18.250000,37.920000
max,24.000000,181.100000


In [166]:
dam_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11160 entries, 0 to 11159
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Delivery Date           11160 non-null  object 
 1   Delivery Hour           11160 non-null  int64  
 2   Repeated Hour Flag      11160 non-null  object 
 3   Settlement Point        11160 non-null  object 
 4   Settlement Point Price  11160 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 436.1+ KB


NOTE: the delivery date is in object type

In [167]:
## Delivery hour:

'''
min = 1    ---------------    00.00 midnight- assume
max = 24   ---------------    23.00 assume

'''

'\nmin = 1    ---------------    00.00 midnight- assume\nmax = 24   ---------------    23.00 assume\n\n'

In [168]:
dam_prices.head(5)

,Delivery Date,Delivery Hour,Repeated Hour Flag,Settlement Point,Settlement Point Price
0,01/01/2022,1,N,HB_BUSAVG,32.72
1,01/01/2022,1,N,HB_HOUSTON,33.20
2,01/01/2022,1,N,HB_HUBAVG,32.69
3,01/01/2022,1,N,HB_NORTH,33.41
4,01/01/2022,1,N,HB_PAN,31.28


In [169]:
## combine the delivery date and hour:

dam_prices['Delivery_Hour_new'] = dam_prices['Delivery Hour'].apply(time_modification)
dam_prices['dateTime'] = pd.to_datetime(dam_prices['Delivery Date'] + ' ' + dam_prices['Delivery_Hour_new'].astype(str) + ':00:00')
dam_prices['dateTime'] = dam_prices['dateTime'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [170]:
dam_prices.head(5)

,Delivery Date,Delivery Hour,Repeated Hour Flag,Settlement Point,Settlement Point Price,Delivery_Hour_new,dateTime
0,01/01/2022,1,N,HB_BUSAVG,32.72,00,2022-01-01 00:00:00
1,01/01/2022,1,N,HB_HOUSTON,33.20,00,2022-01-01 00:00:00
2,01/01/2022,1,N,HB_HUBAVG,32.69,00,2022-01-01 00:00:00
3,01/01/2022,1,N,HB_NORTH,33.41,00,2022-01-01 00:00:00
4,01/01/2022,1,N,HB_PAN,31.28,00,2022-01-01 00:00:00


# 1. Create a csv file containing DAM & RTM prices at hourly intervals from Jan 1 to Dec 31, 2022 for HB_NORTH

In [171]:
dam_prices_HB_NORTH = dam_prices[dam_prices['Settlement Point'] == 'HB_NORTH']
dam_prices_HB_NORTH.head(3)

,Delivery Date,Delivery Hour,Repeated Hour Flag,Settlement Point,Settlement Point Price,Delivery_Hour_new,dateTime
3,01/01/2022,1,N,HB_NORTH,33.41,00,2022-01-01 00:00:00
18,01/01/2022,2,N,HB_NORTH,25.22,01,2022-01-01 01:00:00
33,01/01/2022,3,N,HB_NORTH,22.91,02,2022-01-01 02:00:00


In [172]:
rtm_prices_HB_NORTH = rtm_prices[rtm_prices['Settlement Point'] == 'HB_NORTH']
rtm_prices_HB_NORTH.head(3)

,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point,Settlement Point Price,Delivery_Hour_new,Delivery_interval_new,dateTime
12,01/01/2022,1,1,N,HB_NORTH,53.94,00,0,2022-01-01 00:00:00
13,01/01/2022,1,2,N,HB_NORTH,125.73,00,15,2022-01-01 00:15:00
14,01/01/2022,1,3,N,HB_NORTH,42.02,00,30,2022-01-01 00:30:00


In [173]:
## Hourly RTM price is calculated by averaging the four 15-min interval RTM price for a given hour.

rtm_prices_HB_NORTH['hourly_rtm_price'] = rtm_prices_HB_NORTH.groupby(['Delivery Date', 'Delivery Hour'])['Settlement Point Price'].transform('mean')

<ipython-input-173-ceb30eef3ea9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rtm_prices_HB_NORTH['hourly_rtm_price'] = rtm_prices_HB_NORTH.groupby(['Delivery Date', 'Delivery Hour'])['Settlement Point Price'].transform('mean')


In [174]:
rtm_prices_HB_NORTH.Delivery_interval_new.dtype

dtype('int64')

In [175]:
rtm_prices_HB_NORTH.Delivery_interval_new.value_counts()

0     744
15    744
30    744
45    744
Name: Delivery_interval_new, dtype: int64

In [176]:
rtm_prices_HB_NORTH_filtered = rtm_prices_HB_NORTH[~rtm_prices_HB_NORTH['Delivery_interval_new'].isin([15, 30, 45])]
rtm_prices_HB_NORTH_filtered.head(5)

,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point,Settlement Point Price,Delivery_Hour_new,Delivery_interval_new,dateTime,hourly_rtm_price
12,01/01/2022,1,1,N,HB_NORTH,53.94,00,0,2022-01-01 00:00:00,64.3600
104,01/01/2022,2,1,N,HB_NORTH,32.87,01,0,2022-01-01 01:00:00,32.4475
196,01/01/2022,3,1,N,HB_NORTH,29.91,02,0,2022-01-01 02:00:00,25.5400
288,01/01/2022,4,1,N,HB_NORTH,23.39,03,0,2022-01-01 03:00:00,22.4175
380,01/01/2022,5,1,N,HB_NORTH,8.58,04,0,2022-01-01 04:00:00,15.9475


In [177]:
rtm_prices_HB_NORTH_filtered.shape

(744, 10)

In [178]:
dam_prices_HB_NORTH.shape

(744, 7)

In [179]:
## merged_df = pd.merge(rtm_prices_HB_NORTH_filtered, dam_prices_HB_NORTH, on='dateTime', how='inner')

In [180]:
rtm_prices_HB_NORTH_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 12 to 68368
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Delivery Date           744 non-null    object        
 1   Delivery Hour           744 non-null    int64         
 2   Delivery Interval       744 non-null    int64         
 3   Repeated Hour Flag      744 non-null    object        
 4   Settlement Point        744 non-null    object        
 5   Settlement Point Price  744 non-null    float64       
 6   Delivery_Hour_new       744 non-null    object        
 7   Delivery_interval_new   744 non-null    int64         
 8   dateTime                744 non-null    datetime64[ns]
 9   hourly_rtm_price        744 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 63.9+ KB


In [181]:
dam_prices_HB_NORTH.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 3 to 11148
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Delivery Date           744 non-null    object 
 1   Delivery Hour           744 non-null    int64  
 2   Repeated Hour Flag      744 non-null    object 
 3   Settlement Point        744 non-null    object 
 4   Settlement Point Price  744 non-null    float64
 5   Delivery_Hour_new       744 non-null    object 
 6   dateTime                744 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 46.5+ KB


In [182]:
dam_prices_HB_NORTH['dateTime'] = pd.to_datetime(dam_prices_HB_NORTH['dateTime'])

<ipython-input-182-83744e6cc759>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dam_prices_HB_NORTH['dateTime'] = pd.to_datetime(dam_prices_HB_NORTH['dateTime'])


In [183]:
dam_prices_HB_NORTH.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 3 to 11148
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Delivery Date           744 non-null    object        
 1   Delivery Hour           744 non-null    int64         
 2   Repeated Hour Flag      744 non-null    object        
 3   Settlement Point        744 non-null    object        
 4   Settlement Point Price  744 non-null    float64       
 5   Delivery_Hour_new       744 non-null    object        
 6   dateTime                744 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 46.5+ KB


In [184]:
merged_df = pd.merge(rtm_prices_HB_NORTH_filtered, dam_prices_HB_NORTH, on='dateTime', how='inner')

In [185]:
merged_df.shape

(744, 16)

In [186]:
task1 = merged_df[['dateTime', 'Repeated Hour Flag_y','Settlement Point Price_y','hourly_rtm_price']]
task1.head(5)

,dateTime,Repeated Hour Flag_y,Settlement Point Price_y,hourly_rtm_price
0,2022-01-01 00:00:00,N,33.41,64.3600
1,2022-01-01 01:00:00,N,25.22,32.4475
2,2022-01-01 02:00:00,N,22.91,25.5400
3,2022-01-01 03:00:00,N,18.41,22.4175
4,2022-01-01 04:00:00,N,12.19,15.9475


In [187]:
task1.columns

Index(['dateTime', 'Repeated Hour Flag_y', 'Settlement Point Price_y',
       'hourly_rtm_price'],
      dtype='object')

In [188]:
task1.rename(columns={
    'dateTime': 'date',
    'Repeated Hour Flag_y': 'repeated_hour_flag',
    'Settlement Point Price_y': 'dam',
    'hourly_rtm_price': 'rtm'
}, inplace=True)

<ipython-input-188-4ea5c164f901>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task1.rename(columns={


In [189]:
task1.head(5)

,date,repeated_hour_flag,dam,rtm
0,2022-01-01 00:00:00,N,33.41,64.3600
1,2022-01-01 01:00:00,N,25.22,32.4475
2,2022-01-01 02:00:00,N,22.91,25.5400
3,2022-01-01 03:00:00,N,18.41,22.4175
4,2022-01-01 04:00:00,N,12.19,15.9475


In [190]:
task1.to_csv('task1.csv')

In [191]:
task1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 0 to 743
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                744 non-null    datetime64[ns]
 1   repeated_hour_flag  744 non-null    object        
 2   dam                 744 non-null    float64       
 3   rtm                 744 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 29.1+ KB


In [192]:
task1.describe(include= "all")

<ipython-input-192-3b6c7f5281ce>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  task1.describe(include= "all")


,date,repeated_hour_flag,dam,rtm
count,744,744,744.000000,744.000000
unique,744,1,NaN,NaN
top,2022-01-01 00:00:00,N,NaN,NaN
freq,1,744,NaN,NaN
first,2022-01-01 00:00:00,NaN,NaN,NaN
last,2022-01-31 23:00:00,NaN,NaN,NaN
mean,NaN,NaN,33.650363,30.709153
std,NaN,NaN,14.747701,23.784395
min,NaN,NaN,11.850000,-4.335000
25%,NaN,NaN,24.647500,22.128750


# 2. Create a csv file containing DAM & RTM prices at 15-min intervals from Jan 1 to Dec 31, 2022 for HB_NORTH

In [193]:
rtm_prices['Delivery Date'].value_counts()

01/01/2022    2208
01/17/2022    2208
01/30/2022    2208
01/29/2022    2208
01/28/2022    2208
01/27/2022    2208
01/26/2022    2208
01/25/2022    2208
01/24/2022    2208
01/23/2022    2208
01/22/2022    2208
01/21/2022    2208
01/20/2022    2208
01/19/2022    2208
01/18/2022    2208
01/16/2022    2208
01/02/2022    2208
01/15/2022    2208
01/14/2022    2208
01/13/2022    2208
01/12/2022    2208
01/11/2022    2208
01/10/2022    2208
01/09/2022    2208
01/08/2022    2208
01/07/2022    2208
01/06/2022    2208
01/05/2022    2208
01/04/2022    2208
01/03/2022    2208
01/31/2022    2208
Name: Delivery Date, dtype: int64

December 31 2022 ?? There is no data- assuming one month data only

In [194]:
rtm_prices_HB_NORTH_filtered.head(5)

,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point,Settlement Point Price,Delivery_Hour_new,Delivery_interval_new,dateTime,hourly_rtm_price
12,01/01/2022,1,1,N,HB_NORTH,53.94,00,0,2022-01-01 00:00:00,64.3600
104,01/01/2022,2,1,N,HB_NORTH,32.87,01,0,2022-01-01 01:00:00,32.4475
196,01/01/2022,3,1,N,HB_NORTH,29.91,02,0,2022-01-01 02:00:00,25.5400
288,01/01/2022,4,1,N,HB_NORTH,23.39,03,0,2022-01-01 03:00:00,22.4175
380,01/01/2022,5,1,N,HB_NORTH,8.58,04,0,2022-01-01 04:00:00,15.9475


In [195]:
dam_prices_HB_NORTH.head(5)

,Delivery Date,Delivery Hour,Repeated Hour Flag,Settlement Point,Settlement Point Price,Delivery_Hour_new,dateTime
3,01/01/2022,1,N,HB_NORTH,33.41,00,2022-01-01 00:00:00
18,01/01/2022,2,N,HB_NORTH,25.22,01,2022-01-01 01:00:00
33,01/01/2022,3,N,HB_NORTH,22.91,02,2022-01-01 02:00:00
48,01/01/2022,4,N,HB_NORTH,18.41,03,2022-01-01 03:00:00
63,01/01/2022,5,N,HB_NORTH,12.19,04,2022-01-01 04:00:00


In [196]:
# ## create duplicate rows:
# dam_prices_hb_north_15min = dam_prices_HB_NORTH.loc[dam_prices_HB_NORTH.index.repeat(4)].reset_index(drop=True)
# dam_prices_hb_north_15min

In [197]:
merged_df_task2 = pd.merge(rtm_prices_HB_NORTH, dam_prices_HB_NORTH, on='dateTime', how='outer')
merged_df_task2.head(5)

,Delivery Date_x,Delivery Hour_x,Delivery Interval,Repeated Hour Flag_x,Settlement Point_x,Settlement Point Price_x,Delivery_Hour_new_x,Delivery_interval_new,dateTime,hourly_rtm_price,Delivery Date_y,Delivery Hour_y,Repeated Hour Flag_y,Settlement Point_y,Settlement Point Price_y,Delivery_Hour_new_y
0,01/01/2022,1,1,N,HB_NORTH,53.94,00,0,2022-01-01 00:00:00,64.3600,01/01/2022,1.0,N,HB_NORTH,33.41,00
1,01/01/2022,1,2,N,HB_NORTH,125.73,00,15,2022-01-01 00:15:00,64.3600,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2022,1,3,N,HB_NORTH,42.02,00,30,2022-01-01 00:30:00,64.3600,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2022,1,4,N,HB_NORTH,35.75,00,45,2022-01-01 00:45:00,64.3600,NaN,NaN,NaN,NaN,NaN,NaN
4,01/01/2022,2,1,N,HB_NORTH,32.87,01,0,2022-01-01 01:00:00,32.4475,01/01/2022,2.0,N,HB_NORTH,25.22,01


In [198]:
merged_df_task2['Settlement Point Price_y'].fillna(method='ffill', inplace=True)

In [199]:
merged_df_task2.head(5)

,Delivery Date_x,Delivery Hour_x,Delivery Interval,Repeated Hour Flag_x,Settlement Point_x,Settlement Point Price_x,Delivery_Hour_new_x,Delivery_interval_new,dateTime,hourly_rtm_price,Delivery Date_y,Delivery Hour_y,Repeated Hour Flag_y,Settlement Point_y,Settlement Point Price_y,Delivery_Hour_new_y
0,01/01/2022,1,1,N,HB_NORTH,53.94,00,0,2022-01-01 00:00:00,64.3600,01/01/2022,1.0,N,HB_NORTH,33.41,00
1,01/01/2022,1,2,N,HB_NORTH,125.73,00,15,2022-01-01 00:15:00,64.3600,NaN,NaN,NaN,NaN,33.41,NaN
2,01/01/2022,1,3,N,HB_NORTH,42.02,00,30,2022-01-01 00:30:00,64.3600,NaN,NaN,NaN,NaN,33.41,NaN
3,01/01/2022,1,4,N,HB_NORTH,35.75,00,45,2022-01-01 00:45:00,64.3600,NaN,NaN,NaN,NaN,33.41,NaN
4,01/01/2022,2,1,N,HB_NORTH,32.87,01,0,2022-01-01 01:00:00,32.4475,01/01/2022,2.0,N,HB_NORTH,25.22,01


In [200]:
merged_df_task2.columns

Index(['Delivery Date_x', 'Delivery Hour_x', 'Delivery Interval',
       'Repeated Hour Flag_x', 'Settlement Point_x',
       'Settlement Point Price_x', 'Delivery_Hour_new_x',
       'Delivery_interval_new', 'dateTime', 'hourly_rtm_price',
       'Delivery Date_y', 'Delivery Hour_y', 'Repeated Hour Flag_y',
       'Settlement Point_y', 'Settlement Point Price_y',
       'Delivery_Hour_new_y'],
      dtype='object')

In [201]:
task2_df = merged_df_task2[['dateTime', 'Repeated Hour Flag_x','Settlement Point Price_y','Settlement Point Price_x']]

In [202]:
task2_df.columns

Index(['dateTime', 'Repeated Hour Flag_x', 'Settlement Point Price_y',
       'Settlement Point Price_x'],
      dtype='object')

In [203]:
task2_df.rename(columns={
    'dateTime': 'date',
    'Repeated Hour Flag_x': 'repeated_hour_flag',
    'Settlement Point Price_y': 'dam',
    'Settlement Point Price_x': 'rtm'
}, inplace=True)

<ipython-input-203-df38a7aa31e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task2_df.rename(columns={


In [204]:
task2_df.head(5)

,date,repeated_hour_flag,dam,rtm
0,2022-01-01 00:00:00,N,33.41,53.94
1,2022-01-01 00:15:00,N,33.41,125.73
2,2022-01-01 00:30:00,N,33.41,42.02
3,2022-01-01 00:45:00,N,33.41,35.75
4,2022-01-01 01:00:00,N,25.22,32.87


In [205]:
task2_df.describe(include='all')

<ipython-input-205-43991ba9ba87>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  task2_df.describe(include='all')


,date,repeated_hour_flag,dam,rtm
count,2976,2976,2976.000000,2976.000000
unique,2976,1,NaN,NaN
top,2022-01-01 00:00:00,N,NaN,NaN
freq,1,2976,NaN,NaN
first,2022-01-01 00:00:00,NaN,NaN,NaN
last,2022-01-31 23:45:00,NaN,NaN,NaN
mean,NaN,NaN,33.650363,30.709153
std,NaN,NaN,14.740263,28.427656
min,NaN,NaN,11.850000,-64.200000
25%,NaN,NaN,24.647500,22.217500


In [206]:
task2_df.to_csv('task2.csv')

# 3. Calculate the daily TB2 revenue at HB_NORTH for the following scenarios using the csv files from task
###  Assume that the charge and discharge efficiency is 100%.


In [207]:
task1_df = task1
task1.head(5)

,date,repeated_hour_flag,dam,rtm
0,2022-01-01 00:00:00,N,33.41,64.3600
1,2022-01-01 01:00:00,N,25.22,32.4475
2,2022-01-01 02:00:00,N,22.91,25.5400
3,2022-01-01 03:00:00,N,18.41,22.4175
4,2022-01-01 04:00:00,N,12.19,15.9475


In [208]:
task1_df['rt_tb2'] = (task1_df['rtm'] - task1_df['dam'])
task1_df['da_tb2'] = (task1_df['dam'] - task1_df['rtm'])

<ipython-input-208-c22b53111fab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task1_df['rt_tb2'] = (task1_df['rtm'] - task1_df['dam'])
<ipython-input-208-c22b53111fab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task1_df['da_tb2'] = (task1_df['dam'] - task1_df['rtm'])


In [209]:
task1_df

,date,repeated_hour_flag,dam,rtm,rt_tb2,da_tb2
0,2022-01-01 00:00:00,N,33.41,64.3600,30.9500,-30.9500
1,2022-01-01 01:00:00,N,25.22,32.4475,7.2275,-7.2275
2,2022-01-01 02:00:00,N,22.91,25.5400,2.6300,-2.6300
3,2022-01-01 03:00:00,N,18.41,22.4175,4.0075,-4.0075
4,2022-01-01 04:00:00,N,12.19,15.9475,3.7575,-3.7575
...,...,...,...,...,...,...
739,2022-01-31 19:00:00,N,38.55,25.7850,-12.7650,12.7650
740,2022-01-31 20:00:00,N,36.38,20.1275,-16.2525,16.2525
741,2022-01-31 21:00:00,N,31.88,19.1300,-12.7500,12.7500
742,2022-01-31 22:00:00,N,27.95,18.3575,-9.5925,9.5925


In [210]:
## Assuming that the revenue cannot be negative:

def replace_NEG(x):
    return max(0, x)

task1_df['rt_tb2'] = task1_df['rt_tb2'].apply(replace_NEG).round(2)
task1_df['da_tb2'] = task1_df['da_tb2'].apply(replace_NEG).round(2)


<ipython-input-210-aacfaecc21f7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task1_df['rt_tb2'] = task1_df['rt_tb2'].apply(replace_NEG).round(2)
<ipython-input-210-aacfaecc21f7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task1_df['da_tb2'] = task1_df['da_tb2'].apply(replace_NEG).round(2)


In [211]:
task1_df

,date,repeated_hour_flag,dam,rtm,rt_tb2,da_tb2
0,2022-01-01 00:00:00,N,33.41,64.3600,30.95,0.00
1,2022-01-01 01:00:00,N,25.22,32.4475,7.23,0.00
2,2022-01-01 02:00:00,N,22.91,25.5400,2.63,0.00
3,2022-01-01 03:00:00,N,18.41,22.4175,4.01,0.00
4,2022-01-01 04:00:00,N,12.19,15.9475,3.76,0.00
...,...,...,...,...,...,...
739,2022-01-31 19:00:00,N,38.55,25.7850,0.00,12.76
740,2022-01-31 20:00:00,N,36.38,20.1275,0.00,16.25
741,2022-01-31 21:00:00,N,31.88,19.1300,0.00,12.75
742,2022-01-31 22:00:00,N,27.95,18.3575,0.00,9.59


In [212]:
## group by date

daily_da_tb2 = task1_df.groupby(task1_df['date'].dt.date)['da_tb2'].sum().reset_index()
daily_rt_tb2 = task1_df.groupby(task1_df['date'].dt.date)['rt_tb2'].sum().reset_index()

In [213]:
'''
daily_da_tb2
daily_rt_tb2
'''

'\ndaily_da_tb2\ndaily_rt_tb2\n'

In [214]:
task3_df = pd.merge(daily_da_tb2, daily_rt_tb2, on='date', how='outer')

In [215]:
task3_df

,date,da_tb2,rt_tb2
0,2022-01-01,88.98,51.52
1,2022-01-02,378.33,0.00
2,2022-01-03,217.14,26.81
3,2022-01-04,103.77,12.65
4,2022-01-05,119.92,4.36
5,2022-01-06,75.83,15.74
6,2022-01-07,7.22,878.86
7,2022-01-08,28.94,142.16
8,2022-01-09,190.86,1.91
9,2022-01-10,140.24,51.43


In [216]:
task3_df['date'] = pd.to_datetime(task3_df['date'])
task3_df

,date,da_tb2,rt_tb2
0,2022-01-01,88.98,51.52
1,2022-01-02,378.33,0.00
2,2022-01-03,217.14,26.81
3,2022-01-04,103.77,12.65
4,2022-01-05,119.92,4.36
5,2022-01-06,75.83,15.74
6,2022-01-07,7.22,878.86
7,2022-01-08,28.94,142.16
8,2022-01-09,190.86,1.91
9,2022-01-10,140.24,51.43


In [217]:
task3_df.to_csv('task3.csv')

# 4. Calculate the monthly TB2 revenue in $/kW-month units using the csv files from task for both scenarios at HB_NORTH.


Revenue ($/kW-month)
= Sum of daily revenue ($) / (Battery Power Capacity (kW) x Number of months)


In [218]:
task3_df.head(3)

,date,da_tb2,rt_tb2
0,2022-01-01,88.98,51.52
1,2022-01-02,378.33,0.00
2,2022-01-03,217.14,26.81


In [219]:
## since we need to calculate the TB2 revenue on montly basis
task3_df['month'] = pd.to_datetime(task3_df['date']).dt.to_period('M')
task3_df

,date,da_tb2,rt_tb2,month
0,2022-01-01,88.98,51.52,2022-01
1,2022-01-02,378.33,0.00,2022-01
2,2022-01-03,217.14,26.81,2022-01
3,2022-01-04,103.77,12.65,2022-01
4,2022-01-05,119.92,4.36,2022-01
5,2022-01-06,75.83,15.74,2022-01
6,2022-01-07,7.22,878.86,2022-01
7,2022-01-08,28.94,142.16,2022-01
8,2022-01-09,190.86,1.91,2022-01
9,2022-01-10,140.24,51.43,2022-01


In [220]:
monthly_da_tb2 = task3_df.groupby('month')['da_tb2'].sum().reset_index()
monthly_rt_tb2 = task3_df.groupby('month')['rt_tb2'].sum().reset_index()

In [221]:
monthly_da_tb2, monthly_rt_tb2

(     month   da_tb2
 0  2022-01  4687.45,
      month  rt_tb2
 0  2022-01  2499.2)

In [222]:
battery_power_capacity_kw =  1000 # 1MW
monthly_da_tb2['da_tb2'] = monthly_da_tb2['da_tb2'] / (battery_power_capacity_kw * len(monthly_da_tb2))
monthly_rt_tb2['rt_tb2'] = monthly_rt_tb2['rt_tb2'] / (battery_power_capacity_kw * len(monthly_rt_tb2))

In [223]:
monthly_da_tb2

,month,da_tb2
0,2022-01,4.68745


In [224]:
monthly_rt_tb2

,month,rt_tb2
0,2022-01,2.4992


In [225]:
monthly_tb2_merged = pd.merge(monthly_rt_tb2, monthly_da_tb2, on='month')
monthly_tb2_merged.head(5)

,month,rt_tb2,da_tb2
0,2022-01,2.4992,4.68745


In [226]:
task4_df = monthly_tb2_merged
task4_df.to_csv('task4.csv')

# 5. Calculate the daily TB2 revenue at HB_NORTH using 15-min RTM prices from task Assume that the charge and discharge efficiency is 100%.

In [227]:
battery_power_capacity_kw = 1000  # 1 MW
battery_energy_capacity_kwh = 2000  # 2 MWh
charge_efficiency = 1.0  # 100%
discharge_efficiency = 1.0  # 100%

In [228]:
task2_df

,date,repeated_hour_flag,dam,rtm
0,2022-01-01 00:00:00,N,33.41,53.94
1,2022-01-01 00:15:00,N,33.41,125.73
2,2022-01-01 00:30:00,N,33.41,42.02
3,2022-01-01 00:45:00,N,33.41,35.75
4,2022-01-01 01:00:00,N,25.22,32.87
...,...,...,...,...
2971,2022-01-31 22:45:00,N,27.95,18.17
2972,2022-01-31 23:00:00,N,25.05,18.34
2973,2022-01-31 23:15:00,N,25.05,15.83
2974,2022-01-31 23:30:00,N,25.05,10.88


In [229]:
task2_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2976 entries, 0 to 2975
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                2976 non-null   datetime64[ns]
 1   repeated_hour_flag  2976 non-null   object        
 2   dam                 2976 non-null   float64       
 3   rtm                 2976 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 116.2+ KB
